In [31]:
# import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import filtfilt, butter
#%matplotlib inline

# Function definition 


## Butterworth filter
- digital filter, infinite impulse response
- Butterworth: has maximally flat frequency response in the passband 
- fc: cutoff frequency (Hz)
- fs: sampling frequency (Hz)
- applied using filtfilt:
    - signal twice, once forwards and once backwards
    - order of resulting filter is twice the original filter order


In [32]:
def filter_butter(input_vector, fc, fs, order):
    nyquist = fs/2
    wc = fc / nyquist
    output_vector = np.zeros(input_vector.shape)
    for i in np.arange(3):
        xn = input_vector[:,i]
        b, a = butter(order,wc)
        if not np.all(np.abs(np.roots(a)) < 1):
            print('Filter with cutoff at {} Hz is unstable given ' 'sample frequency {} Hz'.format(cutoff, fs))
        y = filtfilt(b, a, xn)
        output_vector[:,i] = y
    return output_vector

## Derivatives of measurements

- Computes derivatives f_dot = f(t+dt) - f(t)/dt

In [33]:
def get_derivative(measurement, logger_frequency):
    derivatives = np.zeros(measurement.shape)
    derivatives[1:,:]=measurement[1:]-measurement[:-1]
    derivatives[0]=derivatives[1]
    derivatives = derivatives * logger_frequency
    return derivatives

- Computes derivatives f_dot = f(t+dt) - f(t)/dt then filters signal with butterworth filter

In [34]:
def ComputeDotFiltered(input_vector,cutoff_filter,logger_freq, order_filter):
        input_vector_dot = get_derivative(input_vector,logger_freq)
        output_vector = filter_butter(input_vector,cutoff_filter,logger_freq, order_filter)
        return output_vector
    
def ComputeFilteredDotFiltered(input_vector,cutoff_filter,logger_freq, order_filter):
        input_vector_filtered = filter_butter(input_vector,cutoff_filter,logger_freq, order_filter)
        output_vector = ComputeDotFiltered(input_vector_filtered ,cutoff_filter,  logger_freq, order_filter)
        return output_vector

## Fourier series in Python

In [35]:
def fourier_series_q(a, b, axis, N, w_f, q0, time):
    M_a = np.zeros(shape=(N, axis))
    M_b = np.zeros(shape=(N, axis))
    for i in np.arange(axis):
        for j in np.arange(N):
            M_a[j, i] = a[j+(i*2)]
            M_b[j, i] = b[j+(i*2)]
            
    arg = w_f * time
    arg_diff = w_f
    arg_diff_diff = w_f*w_f

    q = np.zeros(shape=(axis))
    dq = np.zeros(shape=(axis))
    ddq = np.zeros(shape=(axis))
    for i in np.arange(axis):
        q[i] += q0[i]
        for j in np.arange(N):
            q[i] += M_a[j,i] * np.sin(arg*(j+1)) + M_b[j,i] * np.cos(arg*(j+1))
    return q

def fourier_series_dq(a, b, axis, N, w_f, q0, time):
    M_a = np.zeros(shape=(N, axis))
    M_b = np.zeros(shape=(N, axis))
    for i in np.arange(axis):
        for j in np.arange(N):
            M_a[j, i] = a[j+(i*2)]
            M_b[j, i] = b[j+(i*2)]
            
    arg = w_f * time
    arg_diff = w_f
    arg_diff_diff = w_f*w_f

    q = np.zeros(shape=(axis))
    dq = np.zeros(shape=(axis))
    ddq = np.zeros(shape=(axis))
    for i in np.arange(axis):
        q[i] = q0[i]
        for j in np.arange(N):
            dq[i] += M_a[j,i] * arg_diff * (j+1) * np.cos(arg*(j+1)) - M_b[j,i] * arg_diff * (j+1) * np.sin(arg*(j+1))      
    return dq

def fourier_series_ddq(a, b, axis, N, w_f, q0, time):
    M_a = np.zeros(shape=(N, axis))
    M_b = np.zeros(shape=(N, axis))
    for i in np.arange(axis):
        for j in np.arange(N):
            M_a[j, i] = a[j+(i*2)]
            M_b[j, i] = b[j+(i*2)]
            
    arg = w_f * time
    arg_diff = w_f
    arg_diff_diff = w_f*w_f

    q = np.zeros(shape=(axis))
    dq = np.zeros(shape=(axis))
    ddq = np.zeros(shape=(axis))
    for i in np.arange(axis):
        q[i] = q0[i]
        for j in np.arange(N):
        
            ddq[i] += - M_a[j,i] * arg_diff_diff * (j+1)*(j+1) * np.sin(arg*(j+1)) - M_b[j,i] * arg_diff_diff * (j+1)*(j+1) * np.cos(arg*(j+1))
            
    return ddq

## Fouriertransform

In [36]:
def fouriertransform(time, data, frequency):
    Fs = frequency
    Ts = 1.0 / Fs
    t = time
    n = len(data[:,0])
    k = np.arange(n)
    T = n/Fs
    frq_two_sides = k/T
    frq_one_side = frq_two_sides[range(int(n/2.0))]
    Y = np.fft.fft(data)/n
    Y = Y[range(int(n/2.0))]
    


## Rotation from IMU frame to Force Sensor frame

In [37]:
def rotate_in_force_sensor_frame(data_in_IMU_frame):
    data_in_FS_frame = np.zeros(data_in_IMU_frame.shape)
    
    data_in_FS_frame[:,0] = data_in_IMU_frame[:,2]
    data_in_FS_frame[:,1] = data_in_IMU_frame[:,0]
    data_in_FS_frame[:,2] = data_in_IMU_frame[:,1]
    
    return data_in_FS_frame


In [38]:
def principal_moments(phi):
    Theta = np.zeros((3,3))
    theta = np.zeros((np.array(phi[:,0:3].shape)))
    for i, number in enumerate(phi[:,0]):
        Theta = np.array([[phi[i,4], phi[i,5],phi[i,6]],[phi[i,5],phi[i,7],phi[i,8]],[phi[i,6],phi[i,8],phi[i,9]]])
        diag = np.diag(Theta)
        theta[i, :] = diag
    return theta

## Objects

### Kinematics Object

In [39]:
class AngularKinematics:
    def __init__(self, avel, aaccel):
        self.avel = avel
        self.aaccel = aaccel

class Kinematics(AngularKinematics):
    def __init__(self, accel, avel, aaccel):
        AngularKinematics.__init__(self, avel, aaccel)
        self.accel = accel
    def computeDerivatives(self, freq):
        self.accel_dot = get_derivative(self.accel,freq)
        self.avel_dot = get_derivative(self.avel,freq)
        self.aaccel_dot = get_derivative(self.aaccel,freq)
    def computeFilteredDerivatives(self,fc, freq, order):    
        computeDerivatives(freq)
        self.filt_accel_dot_filt = ComputeFilteredDotFiltered(self.accel,fc, freq, order)
        self.filt_avel_dot_filt = ComputeFilteredDotFiltered(self.avel,fc, freq, order)
        self.filt_aaccel_dot_filt = ComputeFilteredDotFiltered(self.aaccel,fc,freq, order)
        
        self.accel_dot_filt = ComputeDotFiltered(self.accel,fc, freq, order)
        self.avel_dot_filt = ComputeDotFiltered(self.avel,fc, freq, order)
        self.aaccel_dot_filt = ComputeDotFiltered(self.aaccel,fc, freq, order)
        
class KinematicsData(Kinematics):
    def __init__(self, pos, vel, accel, avel, aaccel):
        Kinematics.__init__(self, accel, avel, aaccel)
        self.pos = pos
        self.vel = vel
    def addGravity(self,g_local):
        self.gravity = g_local
    def addOrientation(self, orientation):
        self.ori = orientation
    def computeDerivatives(self, freq):
        Kinematics.computeDerivatives(self)
        self.pos_dot =  get_derivative(self.pos,freq)
        self.vel_dot = get_derivative(self.vel,freq)
        
    def computeFilteredDerivatives(self, freq, fc, order):
        Kinematics.computeFilteredDerivatives(self, freq, fc, order)
        
        self.filt_pos_dot_filt = ComputeFilteredDotFiltered(self.pos, fc, freq, order)
        self.filt_vel_dot_filt = ComputeFilteredDotFiltered(self.vel, fc, freq, order)
        
        self.pos_dot_filt = ComputeDotFiltered(self.pos,fc, freq, order)
        self.vel_dot_filt = ComputeDotFiltered(self.vel,fc, freq, order)
        
class Input(Kinematics):
    def __init__(self, accel, avel, aaccel, g_local):
        Kinematics.__init__(self, accel, avel, aaccel)
        self.gravity = g_local

class EstimationInput(Input):
    def __init__(self, accel, avel, aaccel, g_local, force, torque):
        Input.__init__(self, accel, avel, aaccel, g_local)
        self.force = force
        self.torque = torque
        
class Estimation(EstimationInput):
    def __init__(self, accel, avel, aaccel, g_local, force, torque, phi):
        EstimationInput.__init__(self, accel, avel, aaccel, g_local, force, torque)
        self.phi = phi
        


        

class FTSensor:
    def __init__(self, force, torque):
        self.force = force
        self.torque = torque
        
class IMUSensor: 
    def __init__(self,accel,avel):
        self.avel = avel
        self.accel = accel

class Sensors (IMUSensor):
    def __init__(self,accel,avel,force, torque):
        IMUSensor.__init__(self,accel,avel)
        self.force = force
        self.torque = torque
    

        
        

            
        
        


### Joint Trajectory Objects

In [40]:
class JointData:
    
    def __init__(self, q, dq):
        self.q = q
        self.dq = dq
        
class JointTrajectory(JointData):
    def __init__(self, q, q_des,dq, dq_des):
        JointData.__init__(self, q, dq)
        self.q_des = q_des
        self.dq_des = dq_des
    def axis(self, axis):
        self.axis = axis
        self.q_axis = self.q[:,7-axis::]
        self.dq_axis = self.dq[:,7-axis::]
    
    def fourier(self, time, a, b, N, fourier_frequency, sampling_frequency, initial_config):    
        if initial_config.shape != self.axis:
            self.q0 = initial_config[7-self.axis::]
        else:
            self.q0 = initial_config 
        self.N = N
        self.a = a
        self.b = b
        self.t = time
        self.q0 *= np.pi/180.0
        self.Ts = 1/sampling_frequency
        self.w_f = fourier_frequency
        
        self.q_fourier = np.array([fourier_series_q(self.a,self.b,self.axis,self.N, self.w_f, self.q0, t) for t in self.t])
        self.dq_fourier = np.array([fourier_series_dq(self.a,self.b,self.axis,self.N, self.w_f, self.q0, t) for t in self.t])
        

        
class JointTrajectorySim(JointTrajectory):
    def __init__(self, q, q_des, dq, dq_des, ddq, ddq_des):
        JointTrajectory.__init__(self, q, q_des, dq, dq_des)
        self.ddq = ddq
        self.ddq_des = ddq_des
        
    def axis(self, axis):
        JointTrajectory.axis(self,axis)
        self.ddq_axis = self.ddq[:,7-axis::]
        
    def fourier(self, time, a, b, N, fourier_frequency, sampling_frequency, initial_config):
        JointTrajectory.fourier(self, time, a, b, N, fourier_frequency, sampling_frequency, initial_config)
        self.ddq_fourier = np.array([fourier_series_ddq(self.a,self.b,self.axis,self.N, self.w_f, self.q0, t) for t in self.t])



### Controller Gain Tuning Objects

In [41]:
class Gains:
    def __init__(self, k_vec):
        self.k = k_vec
class PIDController(Gains):
    def __init__(self, k_vec):
        Gains.__init__(self, k_vec)
        
        self.kp = self.k[0]
        self.kv = self.k[1]
        self.ki = self.k[2]
class JointController(JointTrajectory, PIDController):
    def __init__(self, q, q_des,dq, dq_des, k_vec):
        JointTrajectory.__init__(self, q, q_des,dq, dq_des)
        PIDController.__init__(self,k_vec)

        
class PosController(PIDController):
    def __init__(self, pos_des, pos_curr, vel_des, vel_curr, k_vec_pos):
        PIDController.__init__(self,k_vec_pos)
        self.pos_des = pos_des
        self.pos_curr = pos_curr
        self.vel_des = vel_des
        self.vel_curr = vel_curr       

class OriController:
    def __init__(self, ori_des, ori_curr, avel_des, avel_curr, k_vec_ang):
        PIDController.__init__(self,k_vec_ang)       
        self.ori_des = ori_des
        self.ori_curr =ori_curr
        self.avel_des = vel_des
        self.avel_curr = vel_curr       

class PosOriController(PosController, OriController):
    def __init__(self, pos_des, pos_curr, vel_des, vel_curr, k_vec_pos,ori_des, ori_curr, avel_des, avel_curr, k_vec_ang ):
        PosController.__init__(self, pos_des, pos_curr, vel_des, vel_curr, k_vec_pos)
        OriController.__init__(self, ori_des, ori_curr, avel_des, avel_curr, k_vec_ang)
    

## Plotting Functions
### TUM Colors

In [42]:
blue = (0,0.3961,0.7412)
red = (0.7686,0.0275,0.1059)
green =(0,0.4863,0.1882)
orange =  (0.8902, 0.4471, 0.1333)
purple = (0.4118, 0.0314, 0.3529)
grey = (0.6118, 0.6157, 0.6235)
yellow = (0.9765, 0.7294, 0)
color_list = [blue, red, green, orange, grey, purple, yellow]

### Common labels
#### Object class

In [43]:
class Label:
    def __init__(self, name, abbreviation, unit):
        self.name = name;
        self.abbreviation = abbreviation;
        self.unit = unit

In [44]:
pos_info = Label("Position","$r$", "$m$")
vel_info = Label("Linear  velocity","$v$", r"$\frac{m}{s}$")
accel_info = Label("Linear acceleration", "$a$", r"$\frac{m}{s^2}$")

avel_info = Label("Angular velocity", r"$\omega$", r"$\frac{rad}{s}$")
aaccel_info = Label("Angular acceleration", r"$\alpha$", r"$\frac{rad}{s^2}$")

force_info = Label("Force", r"$F$", r"$N$")
torque_info = Label("Torque", r"$\tau$", r"$Nm$")

frequency_info = Label("Frequency", "$f$" , r"$Hz$")
mass_info = Label("Mass", "$m$", "$kg$")
com_info = Label("Center of mass", "$com$", "$m$")
principal_moments_info = Label("Prinicipal moments of inertia", "$I$", "$kg m^2$")

In [45]:
# class PosOriLabeled(PosoriController, Label):
#     def __init__(self, pos_des, pos_curr, vel_des, vel_curr, k_vec_pos,ori_des, ori_curr, avel_des, avel_curr, k_vec_ang):
        
    

In [46]:
def DataLoading_ValueAssignmentCartesianTrajectory(datafilename, relative_path):
    data_file = datafilename
    path = relative_path
    path_data_file = path + data_file

    file = np.loadtxt(path_data_file,skiprows=2)
    pos_task = PosController()
    pos_des   = file[0::,0:3]    
    pos_curr   = file[0::,3:6]  
    vel_des  = file[0::,6:9]     
    vel_curr  = file[0::,9:12] 
    if (len(file[0,:]))> 12:
        ori_des = file[0::,12:16]
        ori_curr  = file[0::,16:20]
        avel_des = file[0::,20:23]
        avel_curr = file[0::,23:26]
        posori_task = PosOriController(pos_des, pos_curr, vel_des, vel_curr,0, ori_des, ori_curr, avel_des,avel_curr ,0)
        return posori_task
    if (len(file[0,:])) == 12:
        pos_task = PosController(pos_des, pos_curr,0 , vel_des, vel_curr,0 )
        return pos_task


        

#### x-label

In [47]:
x_label = "Time $t$ in $s$"
coordinates = ["$_x$","$_y$","$_z$"]

#### y-label for joint trajectoy

In [48]:
#labels for comparing actual desired

joint_angles = ['$q_1$','$q_2$','$q_3$','$q_4$','$q_5$','$q_6$', '$q_7$']
joint_angles_des = [r'$q_{1_{\mathrm{des}}}$','$q_{2_{\mathrm{des}}}$','$q_{3_{\mathrm{des}}}$','$q_{4_{\mathrm{des}}}$','$q_{5_{\mathrm{des}}}$','$q_{6_{\mathrm{des}}}$', r'$q_{7_{\mathrm{des}}}$']
y_label_joint_angles = "Joint angles $q$ in $rad$"


joint_velocities = ['$\dot{q}_1$','$\dot{q}_2$','$\dot{q}_3$','$\dot{q}_4$','$\dot{q}_5$','$\dot{q}_6$', '$\dot{q}_7$']
joint_velocities_des = ['$\dot{q}_{1_{\mathrm{des}}}$','$\dot{q}_{2_{\mathrm{des}}}$','$\dot{q}_{3_{\mathrm{des}}}$','$\dot{q}_{4_{\mathrm{des}}}$','$\dot{q}_{5_{\mathrm{des}}}$','$\dot{q}_{6_{\mathrm{des}}}$', '$\dot{q}_{7_{\mathrm{des}}}$']
y_label_joint_velocities = r"Joint velocities $\dot{q}$ in $\frac{rad}{s}$" 

joint_accelerations = ['$\ddot{q}_1$','$\ddot{q}_2$','$\ddot{q}_3$','$\ddot{q}_4$','$\ddot{q}_5$','$\ddot{q}_6$', '$\ddot{q}_7$']
joint_accelerations_des = ['$\ddot{q}_{1_{\mathrm{des}}}$','$\ddot{q}_{2_{\mathrm{des}}}$','$\ddot{q}_{3_{\mathrm{des}}}$','$\ddot{q}_{4_{\mathrm{des}}}$','$\ddot{q}_{5_{\mathrm{des}}}$','$\ddot{q}_{6_{\mathrm{des}}}$', '$\ddot{q}_{7_{\mathrm{des}}}$']
y_label_joint_accelerations = r"Joint accelerations $\ddot{q}$ in $\frac{rad}{s^2}$"    


#labels for comparing simulation and hardware
joint_angles_simulation = [r'$q_{1_{\mathrm{simulation}}}$','$q_{2_{\mathrm{simulation}}}$','$q_{3_{\mathrm{simulation}}}$','$q_{4_{\mathrm{simulation}}}$','$q_{5_{\mathrm{simulation}}}$','$q_{6_{\mathrm{simulation}}}$', r'$q_{7_{\mathrm{simulation}}}$']
joint_velocities_simulation = ['$\dot{q}_{1_{\mathrm{simulation}}}$','$\dot{q}_{2_{\mathrm{simulation}}}$','$\dot{q}_{3_{\mathrm{simulation}}}$','$\dot{q}_{4_{\mathrm{simulation}}}$','$\dot{q}_{5_{\mathrm{simulation}}}$','$\dot{q}_{6_{\mathrm{simulation}}}$', '$\dot{q}_{7_{\mathrm{simulation}}}$']
joint_accelerations_simulation = ['$\ddot{q}_{1_{\mathrm{simulation}}}$','$\ddot{q}_{2_{\mathrm{simulation}}}$','$\ddot{q}_{3_{\mathrm{simulation}}}$','$\ddot{q}_{4_{\mathrm{simulation}}}$','$\ddot{q}_{5_{\mathrm{simulation}}}$','$\ddot{q}_{6_{\mathrm{simulation}}}$', '$\ddot{q}_{7_{\mathrm{simulation}}}$']

joint_angles_hardware = [r'$q_{1_{\mathrm{hardware}}}$','$q_{2_{\mathrm{hardware}}}$','$q_{3_{\mathrm{hardware}}}$','$q_{4_{\mathrm{hardware}}}$','$q_{5_{\mathrm{hardware}}}$','$q_{6_{\mathrm{hardware}}}$', r'$q_{7_{\mathrm{hardware}}}$']
joint_velocities_hardware = ['$\dot{q}_{1_{\mathrm{hardware}}}$','$\dot{q}_{2_{\mathrm{hardware}}}$','$\dot{q}_{3_{\mathrm{hardware}}}$','$\dot{q}_{4_{\mathrm{hardware}}}$','$\dot{q}_{5_{\mathrm{hardware}}}$','$\dot{q}_{6_{\mathrm{hardware}}}$', '$\dot{q}_{7_{\mathrm{hardware}}}$']
# joint_accelerations_hardware = ['$\ddot{q}_{1_{\mathrm{hardware}}}$','$\ddot{q}_{2_{\mathrm{hardware}}}$','$\ddot{q}_{3_{\mathrm{hardware}}}$','$\ddot{q}_{4_{\mathrm{hardware}}}$','$\ddot{q}_{5_{\mathrm{hardware}}}$','$\ddot{q}_{6_{\mathrm{hardware}}}$', '$\ddot{q}_{7_{\mathrm{hardware}}}$']


### Plotting function definitions

#### Align y axis when x shared

In [49]:
def align_yaxis(ax1, ax2):
    y_lims = np.array([ax.get_ylim() for ax in [ax1, ax2]])

    # force 0 to appear on both axes, comment if don't need
    y_lims[:, 0] = y_lims[:, 0].clip(None, 0)
    y_lims[:, 1] = y_lims[:, 1].clip(0, None)

    # normalize both axes
    y_mags = (y_lims[:,1] - y_lims[:,0]).reshape(len(y_lims),1)
    y_lims_normalized = y_lims / y_mags

    # find combined range
    y_new_lims_normalized = np.array([np.min(y_lims_normalized), np.max(y_lims_normalized)])

    # denormalize combined range to get new axes
    new_lim1, new_lim2 = y_new_lims_normalized * y_mags
    ax1.set_ylim(new_lim1)
    ax2.set_ylim(new_lim2)

#### Fouriertransfrom

In [50]:
def plot_fourier_transform(time, data, frequency, x_lim_min, x_lim_max):
    
    Fs = frequency
    Ts = 1.0 / Fs
    t = time
    n = len(data)
    k = np.arange(n)
    T = n/Fs
    frq_two_sides = k/T
    frq_one_side = frq_two_sides[range(int(n/2.0))]
    Y = np.fft.fft(data)/n
    Y = Y[range(int(n/2.0))]
    
    plt.figure(figsize=(6,7))

    plt.plot(frq_one_side,abs(Y),c=blue,linewidth=2.0 ) # plotting the spectrum
    plt.grid(True)
    plt.xlabel(frequency_info.name + " "  + frequency_info.abbreviation + " in " + frequency_info.unit)
    plt.xlim([x_lim_min, x_lim_max])
    plt.ylabel("Power Spectral Density")
    


   

In [51]:
def plot_fourier_transform_xyz(time, data, frequency, x_lim_min, x_lim_max,y_lim_max ):
    coordinates = ["$_x$","$_y$","$_z$"]
    Fs = frequency
    Ts = 1.0 / Fs
    t = time
    n = len(data[:,0])
    k = np.arange(n)
    T = n/Fs
    f, axarr = plt.subplots(1,3,figsize=(15,5))
    for idx,ax in enumerate(axarr):
        frq_two_sides = k/T
        frq_one_side = frq_two_sides[range(int(n/2.0))]
        Y = np.fft.fft(data[:,idx])/n
        Y = Y[range(int(n/2.0))]
        ax.plot(frq_one_side,abs(Y), c=color_list[idx], linewidth=2.0, label = frequency_info.abbreviation + str(coordinates[idx])) 
        ax.set_xlabel(frequency_info.name + " "  + frequency_info.abbreviation + " in " + frequency_info.unit)
        ax.set_ylabel("Power Spectral Density")
        ax.set_xlim([x_lim_min, x_lim_max])
        ax.set_ylim([0,y_lim_max])
        ax.grid(True)
        ax.legend(loc=0)
    f.tight_layout()

#### Joint trajectories

In [52]:
def joint_angles_compare(time, axis, angles, angles_des, derivative):
    plt.figure(figsize=(9,7))
    if derivative == 0:
        joint_lables= joint_angles
        joint_lables_des= joint_angles_des
        y_label = y_label_joint_angles
    if derivative == 1:
        joint_lables= joint_velocities
        joint_lables_des= joint_velocities_des
        y_label = y_label_joint_velocities
    if derivative == 2:
        joint_lables= joint_accelerations
        joint_lables_des= joint_accelerations_des
        y_label = y_label_joint_accelerations
    joint_lables = joint_lables[7-axis::]
    joint_lables_des = joint_lables_des[7-axis::]
    for i, lable_q, in enumerate(joint_lables):
        plt.plot(time, angles[:, i], c = color_list[i], label = lable_q, linewidth=2.0 )
        plt.plot(time, angles_des[:, i], c=color_list[i], label = joint_lables_des[i] , linewidth=2.0, linestyle = '--')
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xlim(0, time[-1])
        plt.grid(True)
        plt.legend()
        
def joint_angles_sim_hard(axis, time_sim, angles_sim, time_hardware, angles_hardware, derivative):
    if derivative == 0:
        joint_lables_sim = joint_angles_simulation
        joint_lables_hw = joint_angles_hardware
        y_label = y_label_joint_angles
    if derivative == 1:
        joint_lables_sim = joint_velocities_simulation
        joint_lables_hw = joint_velocities_hardware
        y_label = y_label_joint_velocities    
    joint_lables_sim = joint_lables_sim[7-axis::]
    joint_lables_hw = joint_lables_hw[7-axis::]
    f, axarr = plt.subplots(1,2,figsize=(15,9))
    for i, lable_sim, in enumerate(joint_lables_sim):
        axarr[0].plot(time_sim, angles_sim[:, i], c = color_list[i], label = lable_sim, linewidth=2.0 )
        axarr[0].set_xlim([0, time_sim[-1]])
        axarr[0].set_ylabel(y_label)
        axarr[0].grid(True)
        axarr[0].legend()
    for i, lable_hw in enumerate(joint_lables_hw):
        axarr[1].plot(time_hardware, angles_hardware[:, i], c = color_list[i], label = lable_hw, linewidth=2.0 )
        axarr[1].set_xlim([0, time_hardware[-1]])
        axarr[1].set_ylabel(y_label)
        axarr[1].grid(True)
        axarr[1].legend()
    axarr[0].set_xlabel(x_label)
    axarr[1].set_xlabel(x_label)

#### Three dimensional variables

In [53]:
def xyz(time, data, variable_name, variable_abbreviation, variable_unit):
    plt.figure(figsize=(9,7))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx, coordinate in enumerate(coordinates):
        plt.plot(time, data[:,idx], c = color_list[idx], label = variable_abbreviation + str(coordinate),  linewidth=2.0)
        plt.xlabel(x_label)
        plt.ylabel(variable_name + " " + variable_abbreviation + " in " + variable_unit)
        plt.xlim(0, time[-1])
        plt.grid(True)
        plt.legend()
        
def xyz_obj(time, data, object_info):
    plt.figure(figsize=(9,7))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx, coordinate in enumerate(coordinates):
        plt.plot(time, data[:,idx], c = color_list[idx], label = object_info.abbreviation + str(coordinate),  linewidth=2.0)
        plt.xlabel(x_label)
        plt.ylabel(object_info.name + " " + object_info.abbreviation + " in " + object_info.unit)
        plt.xlim(0, time[-1])
        plt.grid(True)
        plt.legend()

def xyz_subplots_three_inputs_two_axes(time, data_1, data_2, data_3, object_info1,object_info2,label_list):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,ax in enumerate(axarr):
        ax2 = ax.twinx()
        ax2.plot(time, data_3[:,idx], c=color_list[2], linewidth=2.0, label = object_info2.abbreviation + str(coordinates[idx]) + " " + label_list[1])
        ax.plot(time, data_1[:,idx], c=color_list[0], linewidth=2.0, label = object_info1.abbreviation + str(coordinates[idx]) )
        ax.set_xlim([0, time[-1]])
        ax.set_ylabel(object_info1.name + " " + object_info1.abbreviation + " in " + object_info1.unit)
        ax.grid(True)
        ax.set_xlabel(x_label)
        ax2.plot(time, data_2[:,idx], c=color_list[1], linewidth=2.0, label = object_info2.abbreviation + str(coordinates[idx]) + " " + label_list[0]) 
        ax2.set_ylabel(object_info2.name + " " + object_info2.abbreviation + " in " + object_info2.unit)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc=0)
        align_yaxis(ax, ax2)
    f.tight_layout()
    
def xyz_subplots_two_inputs_two_axes(time, data_1, data_2, object_info1,object_info2 ,label_list):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,ax in enumerate(axarr):
        ax2 = ax.twinx()
        ax.plot(time, data_1[:,idx], c=color_list[0], linewidth=2.0, label = object_info1.abbreviation + str(coordinates[idx])+ " " + label_list[0])
        ax.set_xlim([0, time[-1]])
        ax.set_ylabel(object_info1.name + " " + object_info1.abbreviation + " in " + object_info1.unit)
        ax.grid(True)
        ax.set_xlabel(x_label)
        ax2.plot(time, data_2[:,idx], c=color_list[1], linewidth=2.0, label = object_info2.abbreviation + str(coordinates[idx]) + " " + label_list[1]) 
        ax2.set_ylabel(object_info2.name + " " + object_info2.abbreviation + " in " + object_info2.unit)
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc=0)
        align_yaxis(ax, ax2)
    f.tight_layout()
    
def xyz_subplots_two_inputs(time, data_1, data_2, object_info,label_list):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,ax in enumerate(axarr):
        ax.plot(time, data_1[:,idx], c=color_list[0], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[0])
        ax.set_xlim([0, time[-1]])
        ax.set_ylabel(object_info.name + " " + object_info.abbreviation + " in " + object_info.unit)
        ax.set_xlabel(x_label)
        ax.grid(True)
        ax.plot(time, data_2[:,idx], c=color_list[1], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[1]) 
        lines, labels = ax.get_legend_handles_labels()
        ax.legend(lines , labels , loc=0)
    f.tight_layout()
    
def xyz_subplots_three_inputs(time, data_1, data_2,data_3, object_info,label_list):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,ax in enumerate(axarr):
        ax.plot(time, data_1[:,idx], c=color_list[0], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[0])
        ax.plot(time, data_2[:,idx], c=color_list[1], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[1]) 
        ax.plot(time, data_3[:,idx], c=color_list[2], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[2]) 
        ax.set_xlim([0, time[-1]])
        ax.set_ylabel(object_info.name + " " + object_info.abbreviation + " in " + object_info.unit)
        ax.set_xlabel(x_label)
        lines, labels = ax.get_legend_handles_labels()
        ax.grid(True)
        ax.legend(lines , labels , loc=0)
    f.tight_layout()
    
# def xyz_subplots_four_inputs(time, data_1, data_2,data_3, data_4, object_info,label_list):
def xyz_subplots_four_inputs(time, data_1, data_2,data_3, data_4, object_info,label_list):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,ax in enumerate(axarr):
        ax.plot( data_1[:,idx], c=color_list[0], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[0])
        ax.plot( data_2[:,idx], c=color_list[1], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[1]) 
        ax.plot( data_3[:,idx], c=color_list[2], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[2]) 
        ax.plot( data_4[:,idx], c=color_list[3], linewidth=2.0, label = object_info.abbreviation + str(coordinates[idx]) + " " + label_list[3]) 
#         ax.set_xlim([0, time[-1]])
        ax.set_ylabel(object_info.name + " " + object_info.abbreviation + " in " + object_info.unit)
        ax.set_xlabel(x_label)
        lines, labels = ax.get_legend_handles_labels()
        ax.grid(True)
        ax.legend(lines , labels , loc=0)
    f.tight_layout()
    

#### Estimated inertial parameters

In [54]:
def Plot_intertial_params_one_error(time, data,  real_values):
    f, axarr = plt.subplots(10,1,figsize=(6,15))
    params = ['Mass in $kg$', 'COM_x in $m$','COM_y in $m$', 'COM_z in $m$', '$I_{xx}$','$I_{xy}$', '$I_{xz}$','$I_{yy}$','$I_{yz}$','$I_{zz}$']
    if 0 in data[:,0]: 
        data[:,1] = 0
        data[:,2] = 0
        data[:,3] = 0
    else:
        data[:,1] /= data[:,0]
        data[:,2] /= data[:,0]
        data[:,3] /= data[:,0]
    mse = np.empty_like(data)
    for idx,value in enumerate(real_values):                           #for all 10 inertial params  
        for i in np.arange(np.size(data[:,0])):                       #elementwise 
            mse[i,idx] = (np.square(data[i,idx] - value)).mean(axis=None)
    for idx, param in enumerate(params):
        axarr[idx].plot(time, mse[:,idx], c=blue)
        axarr[idx].set_title(param)
        axarr[idx].set_xlim([0, time[-1]])
    axarr[9].set_xlabel(x_label)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
    plt.tight_layout()
    
def Plot_intertial_params_one(time, data, real_values):
    f, axarr = plt.subplots(4,1,figsize=(6,15))
    params = ['Mass in $kg$', 'COM_x in $m$','COM_y in $m$', 'COM_z in $m$', '$I_{xx}$','$I_{xy}$', '$I_{xz}$','$I_{yy}$','$I_{yz}$','$I_{zz}$']
    if 0 in data[:,0]: 
        data[:,1] = 0
        data[:,2] = 0
        data[:,3] = 0
    else:
        data[:,1] /= data[:,0]
        data[:,2] /= data[:,0]
        data[:,3] /= data[:,0]
    mass = data[:,0]
    com = data[:,1:4]
    com_label = ["$c_x$","$c_y$","$c_z$"]
    com_real = np.arange(real_values[1],real_values[2],real_values[3])
    princip = principal_moments(phi_RLS) 
    princip_real = np.arange(real_values[4],real_values[7],real_values[9])
    princip_label = ["$I_{xx}$","$I_{yy}$","$I_{zz}$"]
    for index, number in enumerate(data[:,0]):
        if number != 0:
            x_lim_min = index/1000
            break
    for idx in np.arange(3):
        axarr[idx].set_title(params[idx])
        axarr[idx].set_xlim([x_lim_min, time[-1]])
        axarr[idx].axhline(real_values[idx], c=red)
    axarr[0].plot(time, mass, c = blue)
    axarr[0].set_ylabel(mass_info.name + " " + mass_info.abbreviation + " in " + mass_info.unit)
    axarr[0].axhline(real_values[0], c = blue , linestyle = "--")
    for i in np.arange(3):
        axarr[1].plot(time, com[:,i], c = color_list[i], label = com_info.abbreviation + str(coordinates[i]),linewidth=2.0)
        axarr[1].axhline(com_real[i], c=color_list[i], linestyle = "--")
        axarr[2].plot(time, princip[:,i], c = color_list[i], label = princip_label[i],linewidth=2.0)
        axarr[2].axhline(princip_real[i], c=color_list[i], linestyle = "--")
    axarr[1].set_ylabel(com_info.name + " " + com_info.abbreviation + " in " + com_info.unit)
    axarr[2].set_ylabel(principal_moments_info.name + " " + principal_moments_info.abbreviation + " in " + principal_moments_info.unit)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)

#### Comparing Simulation and hardware

In [55]:
def sim_hard(time_sim, data_sim, time_hardware, data_hardware,variable_name, variable_abbreviation, variable_unit,label_1, label_2 , plot_quantity):
    if plot_quantity ==1:
        f, axarr = plt.subplots(1,2,figsize=(15,9))
        coordinates = ["$_x$","$_y$","$_z$"]
        for idx,coordinate in enumerate(coordinates):
            axarr[0].plot(time_sim, data_sim[:,idx], c=color_list[idx], linewidth=2.0, label = variable_abbreviation + str(coordinate) + " " + label_1)
            axarr[0].set_xlim([0, time_sim[-1]])
            axarr[0].set_ylabel(variable_name + " " + variable_abbreviation + " in " + variable_unit)
            axarr[0].grid(True)
            axarr[0].legend()
        for idx,coordinate in enumerate(coordinates):
            axarr[1].plot(time_hardware, data_hardware[:,idx], c=color_list[idx], linewidth=2.0, label = variable_abbreviation + str(coordinate) + " " + label_2)
            axarr[1].set_xlim([0, time_hardware[-1]])
            axarr[1].set_ylabel(variable_name + " " + variable_abbreviation + " in " + variable_unit)
            axarr[1].grid(True)
            axarr[1].legend()
        axarr[0].set_xlabel(x_label)
        axarr[1].set_xlabel(x_label)
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        
    if plot_quantity == 3:
        f, axarr = plt.subplots(3,2,figsize=(15,9))
        coordinates = ["$_x$","$_y$","$_z$"]
        for idx,coordinate in enumerate(coordinates):
            axarr[idx,0].plot(time_sim, data_sim[:,idx], c=color_list[idx], linewidth=2.0, label = variable_abbreviation + str(coordinate) + " " + label_1)
            axarr[idx,0].set_xlim([0, time_sim[-1]])
            axarr[idx,0].set_ylabel(variable_name + " " + variable_abbreviation + " in " + variable_unit)
            axarr[idx,0].grid(True)
            axarr[idx,0].legend()
        for idx,coordinate in enumerate(coordinates):
            axarr[idx,1].plot(time_hardware, data_hardware[:,idx], c=color_list[idx], linewidth=2.0, label = variable_abbreviation + str(coordinate) + " " + label_2)
            axarr[idx,1].set_xlim([0, time_hardware[-1]])
            axarr[idx,1].set_ylabel(variable_name + " " + variable_abbreviation + " in " + variable_unit)
            axarr[idx,1].grid(True)
            axarr[idx,1].legend()
        axarr[2,0].set_xlabel(x_label)
        axarr[2,1].set_xlabel(x_label)
        plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False)
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)

#### Force and Torque

In [56]:
def force_torque(time, data_1, data_2):
    f, axarr = plt.subplots(3,2,figsize=(15,9))
    coordinates = ["$_x$","$_y$","$_z$"]
    for idx,coordinate in enumerate(coordinates):
        axarr[idx,0].plot(time, data_1[:,idx], c=color_list[idx], linewidth=2.0)
        axarr[idx,0].set_xlim([0, time[-1]])
        axarr[idx,0].set_ylabel("$F$"+str(coordinate)+" in " "$N$")
        axarr[idx,0].grid(True)
    for idx,coordinate in enumerate(coordinates):
        axarr[idx,1].plot(time, data_2[:,idx], c=color_list[idx], linewidth=2.0)
        axarr[idx,1].set_xlim([0, time[-1]])
        axarr[idx,1].set_ylabel(r"$\tau$"+str(coordinate)+" in " "$Nm$")
        axarr[idx,1].grid(True)
    axarr[2,0].set_xlabel(x_label)
    axarr[2,1].set_xlabel(x_label)
    plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)

#### Gain Tuning

In [57]:
def plot(time, info, *args):
    f, axarr = plt.subplots(3,1,figsize=(15,15))        
    for ax in axarr:
        for idx, data in enumerate(*args):
            ax.plot(time, data[:,idx], c=color_list[0], linewidth=2.0, label = "desired")
            ax.plot(time, data[:,idx], c=color_list[1], linewidth=2.0, label = "current, kp = "+ controller.kp_pos +", kv = "+ controller.kv_pos + " ki = "+ controller.ki_pos) 
            ax.set_xlim([0, time[-1]])
            ax.set_ylabel(object_info.name + " " + object_info.abbreviation + " in " + object_info.unit)
            ax.set_xlabel(x_label)
            lines, labels = ax.get_legend_handles_labels()
            ax.grid(True)
            ax.legend(lines , labels , loc=0)
    f.tight_layout()

In [58]:
def posori_plot(time,var_type, *args):
    f, axarr = plt.subplots(3,1,figsize=(15,15))
    data_label = ["desired", "current, kp = "+ controller.kp_pos +", kv = "+ controller.kv_pos + " ki = "+ controller.ki_pos]
    if var_type == "pos":
        
        for ax in axarr:
            for idx, controller in enumerate(*args):
                ax.plot(time, controller.pos_des[:,idx], c=color_list[0], linewidth=2.0, label = data_label[0])
                ax.plot(time, controller.pos_curr[:,idx], c=color_list[1], linewidth=2.0, label = data_label[1]) 
                ax.set_xlim([0, time[-1]])
                ax.set_ylabel(pos_info.name + " " +pos_info.abbreviation + " in " + pos_info.unit)
                ax.set_xlabel(x_label)
                lines, labels = ax.get_legend_handles_labels()
                ax.grid(True)
                ax.legend(lines , labels , loc=0)
        f.tight_layout()
    if var_type == "vel":

        for ax in axarr:
            for idx, controller in enumerate(*args):
                ax.plot(time, controller.vel_des[:,idx], c=color_list[0], linewidth=2.0, label = data_label[0])
                ax.plot(time, controller.vel_curr[:,idx], c=color_list[1], linewidth=2.0, label = data_label[1]) 
                ax.set_xlim([0, time[-1]])
                ax.set_ylabel(pos_info.name + " " +pos_info.abbreviation + " in " + pos_info.unit)
                ax.set_xlabel(x_label)
                lines, labels = ax.get_legend_handles_labels()
                ax.grid(True)
                ax.legend(lines , labels , loc=0)
        f.tight_layout()
    

    